In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !nvidia-smi

In [3]:
# pip install tensorflow gpu

In [4]:
# import tensorflow as tf

# # Set memory growth to avoid allocating all GPU memory at once
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
            
#         # Set per_process_gpu_memory_fraction
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*4)]  # Memory limit in MB
#         )
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)
# else:
#     print("No GPU devices found")

# # Verify configuration
# print("Physical devices:", tf.config.experimental.list_physical_devices())
# print("Logical devices:", tf.config.experimental.list_logical_devices())


In [5]:
import tensorflow as tf  # Import TensorFlow library
import pandas as pd  # Import pandas for handling CSV files
import os  # Import os module for file and directory operations
import sys  # Import sys module for argument parsing
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping
import shutil  # Import shutil for zipping files
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Function to load images
def load_images(image_paths, target_size=(255, 255)):
    images = []  # Initialize an empty list to store images
    for img_path in image_paths:
        img = load_img(img_path, target_size=target_size)  # Load image and resize
        img_array = img_to_array(img)  # Convert image to array
        img_array = img_array / 255.0  # Normalize the image to [0, 1]
        images.append(img_array)  # Append the image to the list
    return tf.convert_to_tensor(images, dtype=tf.float32)  # Convert list of images to a tensor

# Function to load data from a given directory and CSV file
def load_data(images_dir, csv_file):
    data = pd.read_csv(csv_file)  # Read the CSV file into a DataFrame
    print("CSV Columns:", data.columns)  # Print the columns in the CSV file to debug the issue

    # Print the first few rows to check the data
    print("First few rows of the data:\n", data.head())

    # Check for missing values in the DataFrame
    if data.isnull().values.any():
        print("Data contains missing values. Please check the CSV file.")
        print(data.isnull().sum())  # Print count of missing values in each column
        return None, None, None, None

    # Create full image paths by joining directory and image filename from CSV
    image_paths = [os.path.join(images_dir, img_path) for img_path in data['filename'].values]

    # Map string labels to numerical labels
    class_mapping = {
        'car': 0, 'person': 1, 'traffic light': 2, 'red light': 3, 'stop': 4,
        'motorbike': 5,'pole': 6, 'edge lane': 7, 'middle lane': 8, 'truck': 9,
        'bicycle': 10, 'walk way': 11, 'bus': 12,'Straight and Right Arrow': 13,
        'straight and left arrow': 14, 'Green light': 15, 'lanes': 16,
        'road': 17,'90': 18, 'bench': 19, 'Yellow light': 20, 'StraightArrow':
        21, 'pottedplant': 22, 'left arrow': 23,'Right Arrow': 24, 'bird': 25,
        '60': 26,'barrie': 27,'briage': 28, 'motor': 29,
      }

    # Print unique classes in 'class' column
    print("Unique classes in the dataset:", data['class'].unique())

    labels = data['class'].map(class_mapping).values  # Convert string labels to numerical labels

    # Check for NaN values in labels
    if pd.isnull(labels).any():
        print("Found NaN values in the class column. Please check your class_mapping.")
        print(data[pd.isnull(labels)])  # Print rows with NaN values
        return None, None, None, None

    # Print unique classes in labels
    print("Unique classes in labels after mapping:", pd.unique(labels))

    num_classes = len(class_mapping)  # Determine the number of unique classes
    labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)  # Convert labels to one-hot encoding

    # Prepare bounding box coordinates and other information
    bboxes = data[['xmin', 'ymin', 'xmax', 'ymax']].values  # Extract bounding box coordinates
    images = load_images(image_paths)  # Load images using the load_images function
    return images, labels, bboxes, num_classes  # Return images, labels, bounding boxes, and number of classes

# Load training data
train_images_dir = r"C:\Users\Admin\Documents\ANN\Carla Dataset\train"  # Directory containing training images
train_csv_file = r"C:\Users\Admin\Documents\ANN\Carla Dataset\train\_annotations.csv"  # Update this path to point to your actual CSV file
x_train, y_train, _, num_classes = load_data(train_images_dir, train_csv_file)

# Check if training data loaded successfully
if x_train is None or y_train is None or num_classes is None:
    print("Failed to load training data. Please check the dataset and the load_data function.")
    sys.exit(1)  # Exit if data loading failed

# Load validation data
valid_images_dir = r"C:\Users\Admin\Documents\ANN\Carla Dataset\valid"  # Directory containing validation images
valid_csv_file = r"C:\Users\Admin\Documents\ANN\Carla Dataset\valid\_annotations.csv"  # Update this path as well
x_valid, y_valid, _, _ = load_data(valid_images_dir, valid_csv_file)  # Load validation data

# Check if validation data loaded successfully
if x_valid is None or y_valid is None:
    print("Failed to load validation data. Please check the dataset and the load_data function.")
    sys.exit(1)  # Exit if data loading failed

# Load test data
# test_images_dir = "/content/drive/MyDrive/Dataset/Carla-Dataset/test"  # Directory containing test images
# test_csv_file = "/content/drive/MyDrive/Dataset/Carla-Dataset/test/_annotations.csv"  # Update this path as well
# x_test, y_test, _, _ = load_data(test_images_dir, test_csv_file)  # Load test data

# # Check if test data loaded successfully
# if x_test is None or y_test is None:
#     print("Failed to load test data. Please check the dataset and the load_data function.")
#     sys.exit(1)  # Exit if data loading failed

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(x_train)

# Create a Convolutional neural network
# Corrected model creation
model = tf.keras.models.Sequential()

# Add layers to the model
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(255, 255, 3)))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Add more layers as needed
model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(128, (3, 3)))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(num_classes))
model.add(tf.keras.layers.Activation("softmax"))


# Compile the model
history = model.compile(optimizer="adam",
              # loss="sparse_categorical_crossentropy",,
              # loss="squared_hinge",
              # loss="hinge",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Define callbacks
log_dir = r"C:\Users\Admin\Documents\ANN\Carla Dataset\logs"  # Directory to save TensorBoard logs

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.1000),
    # EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    TensorBoard(log_dir=log_dir)
]

# Train the model
model.fit(
    datagen.flow(x_train, y_train, batch_size=3),  # Use a smaller batch size
    epochs=30,
    validation_data=(x_valid, y_valid),
    callbacks=callbacks
)

# Evaluate the model
# test_loss, test_acc = model.evaluate(x_test, y_test)
# print("Test accuracy:", test_acc)

# Save the trained model
# Set the directory to my Google Drive
drive_folder_path = r'C:\Users\Admin\Documents\ANN\Carla Dataset\saved_model'
os.makedirs(drive_folder_path, exist_ok=True)

# Saving the model in the native Keras format
keras_filename = os.path.join(drive_folder_path, "saved_model.keras")
model.save(keras_filename)  # Save the model to .keras file
print(f"Model saved to {keras_filename}.")

# # Zipping the directory containing the saved model
# shutil.make_archive(drive_folder_path, 'zip', drive_folder_path)
# print(f"Model directory zipped to {drive_folder_path}.zip")

CSV Columns: Index(['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'], dtype='object')
First few rows of the data:
                                             filename  width  height   class  \
0  rgb_20240622112419726764_jpg.rf.21f698cc5c1740...    255     255     bus   
1  rgb_20240622112419726764_jpg.rf.21f698cc5c1740...    255     255     car   
2  rgb_20240622113116438046_jpg.rf.21f988c49c5be0...    255     255    stop   
3  rgb_20240622112540883784_jpg.rf.221406a6ff8080...    255     255  person   
4  rgb_20240622112540883784_jpg.rf.221406a6ff8080...    255     255     car   

   xmin  ymin  xmax  ymax  
0   124   108   191   152  
1   212   127   253   153  
2    51   170   190   223  
3   125   121   132   152  
4    54   125    65   133  
Unique classes in the dataset: ['bus' 'car' 'stop' 'person' 'truck' 'edge lane' 'lanes' 'traffic light'
 'pole' 'Green light' 'middle lane' 'StraightArrow' 'bicycle' 'red light'
 'motorbike' 'road' 'Straight and Right A

In [6]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Check if 'val_accuracy' and 'val_loss' exist in history
    has_val_data = 'val_accuracy' in history.history and 'val_loss' in history.history

    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 6))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    if has_val_data:
        plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train'] + (['Validation'] if has_val_data else []), loc='upper left')

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    if has_val_data:
        plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train'] + (['Validation'] if has_val_data else []), loc='upper left')

    plt.tight_layout()
    plt.savefig('training_history.png')  # Save the figure as a .png file
    plt.show()

# Plot the training history
plot_training_history(history)


AttributeError: 'NoneType' object has no attribute 'history'

In [ ]:
from google.colab import files

files.download('training_history.png')

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to visualize predictions with bounding boxes
def predict_and_visualize(model, x_test, y_test, bboxes_test, class_mapping, num_samples=5):
    # Predict the class probabilities for the test images
    y_pred_prob = model.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Get the reverse mapping from numerical labels to class names
    reverse_class_mapping = {v: k for k, v in class_mapping.items()}

    for i in range(num_samples):
        img = x_test[i].numpy() if isinstance(x_test[i], tf.Tensor) else x_test[i]
        true_label = reverse_class_mapping[y_true[i]]
        predicted_label = reverse_class_mapping[y_pred[i]]
        bbox = bboxes_test[i]

        fig, ax = plt.subplots(1)
        ax.imshow(img)

        # Create a rectangle patch for the bounding box
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1], linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Display true and predicted labels
        plt.title(f"True: {true_label}, Predicted: {predicted_label}")
        plt.show()

# Example usage
# Assuming x_test, y_test, and bboxes_test are defined
# Example:
# x_test = tf.convert_to_tensor([...])  # Replace [...] with actual test images
# y_test = tf.convert_to_tensor([...])  # Replace [...] with actual test labels (one-hot encoded)
# bboxes_test = np.array([...])  # Replace [...] with actual bounding boxes

# Define class mapping
class_mapping = {
    'car': 0, 'person': 1, 'traffic light': 2, 'red light': 3, 'stop': 4,
    'motorbike': 5, 'pole': 6, 'edge lane': 7, 'middle lane': 8, 'truck': 9,
    'bicycle': 10, 'walk way': 11, 'bus': 12, 'Straight and Right Arrow': 13,
    'straight and left arrow': 14, 'Green light': 15, 'lanes': 16,
    'road': 17, '90': 18, 'bench': 19, 'Yellow light': 20, 'StraightArrow':
    21, 'pottedplant': 22, 'left arrow': 23, 'Right Arrow': 24, 'bird': 25,
    '60': 26, 'barrie': 27, 'bridge': 28, 'motor': 29
}

# Call the function with example data
predict_and_visualize(model, x_test, y_test, bboxes_test, class_mapping, num_samples=5)
